<div>
    <img src='../pics/banner.PNG'/>
</div>
<div>
    <img width="10%" height="10%" src='../pics/python-programming.png'/>
    <img src='../pics/miw.PNG'/>
</div>
<div>
    <em>Author: Jeroen Boogaard</em>
</div>

<h1>Practicum SQL </h1>

<h2>Imports<h2>

In [1]:
from os import system as sys
sys('pip install pysqlite3 --upgrade pip')

1

In [2]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
from PIL import Image
import pandas as pd
import sqlite3

<h2>Extract, Transform and Load</h2>

<h3>Extract</h3>

In [3]:
# https://raw.githubusercontent.com/ajturner/acetate/master/places/Countries-Europe.csv
csvFile = "../csv/countries_europe.csv"
euCsvFile = "../csv/european_union.csv"

 # Create Pandas DataFrames
europeDF = pd.read_csv(csvFile)
euDF = pd.read_csv(euCsvFile)
print('Dit is europeDF: ')
print(europeDF)
print('Dit is euDF: ')
print(euDF)

Dit is europeDF: 
    zoom                    name  abbreviation Code  ISO numeric   
0      3                 Ukraine          Ukr.  UKR          804  \
1      3                  France           Fr.  FRA          250   
2      3                   Spain         Spain  ESP          724   
3      3                  Sweden          Swe.  SWE          752   
4      3                 Germany          Ger.  DEU          276   
5      3                 Finland          Fin.  FIN          246   
6      3                  Norway          Nor.  NOR          578   
7      3                  Poland          Pol.  POL          616   
8      3                   Italy         Italy  ITA          380   
9      3          United Kingdom          U.K.  GBR          826   
10     3                 Romania          Rom.  ROU          642   
11     3                 Belarus         Bela.  BLR          112   
12     3                  Greece          Grc.  GRC          300   
13     3                Bulgar

<h3>Transform</h3>

<h3>Exercise 1</h3>
<h4>Gegeven</h4>

In [4]:
benelux = {"Belgium", "Netherlands", "Luxembourg"}
nordics = {"Denmark", "Finland", "Iceland", "Norway"}
baltics = {"Estonia", "Latvia", "Lithuania"}

<h4>Gevraagd</h4>
<ol>
    <li>Voeg de kolom <u>group</u> toe aan de dataset <u>europeDF</u> met daarin de groep waartoe een land behoort volgens bovenstaande sets</li>
    <li>Ga na of er nog meer landengroepen zijn binnen Europa en voeg die ook toe</li>
    <li>Sla deze verrijkte dataset op in een nieuw CSV bestand</li>
</ol>

<h4>Oplossing</h4>

In [6]:
# Oplossing
# first add empty column 'group' to dataframe, then fill that column with given values
europeDF["groups"] = ""

#https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
#df.loc[df[<some_column_name>] == <condition>, [<another_column_name>]] = <value_to_add>
for i in benelux:
    europeDF.loc[europeDF['name'] == i, ['groups']] = 'benelux'
for i in nordics:
    europeDF.loc[europeDF['name'] == i, ['groups']] = 'nordics'
for i in baltics:
    europeDF.loc[europeDF['name'] == i, ['groups']] = 'baltics'
    
print(europeDF)

    zoom                    name  abbreviation Code  ISO numeric   
0      3                 Ukraine          Ukr.  UKR          804  \
1      3                  France           Fr.  FRA          250   
2      3                   Spain         Spain  ESP          724   
3      3                  Sweden          Swe.  SWE          752   
4      3                 Germany          Ger.  DEU          276   
5      3                 Finland          Fin.  FIN          246   
6      3                  Norway          Nor.  NOR          578   
7      3                  Poland          Pol.  POL          616   
8      3                   Italy         Italy  ITA          380   
9      3          United Kingdom          U.K.  GBR          826   
10     3                 Romania          Rom.  ROU          642   
11     3                 Belarus         Bela.  BLR          112   
12     3                  Greece          Grc.  GRC          300   
13     3                Bulgaria          Blg.  

In [ ]:
balkan = {"Albania", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Kosovo", "Moldova", "Montenegro", "North Macedonia", "Romania", "Serbia", "Slovenia"}
mediterranian = {"Spain", "France", "Monaco", "Italy", "Malta", "Slovenia", "Croatia",  "Bosnia and Herzegovina", "Montenegro", "Albania", "Greece"} 
for i in mediterranian:
    europeDF.loc[europeDF['name'] == i, ['groups']] = 'balkan'
for i in balkan:
    europeDF.loc[europeDF['name'] == i, ['groups']] = 'mediterranean'
print(europeDF)
# Conclusie: als 1 land (bv Albanie) in 2 groepen valt (mediterrenean en balkan) dan komt alleen de groepsnaam van de laatst ingelezen set in de group-kolom, de eerste  group-naam wordt overschreven

<h3>Load</h3>

<p>Maak een lege database</p>

In [7]:
sys('sqlite3 ../db/europe.db "VACUUM;"')

1

In [8]:
# Connect to SQLite database
dbConnection = sqlite3.connect('../db/europe.db')

OperationalError: unable to open database file

<p>Maak tabellen o.b.v. de dataset</p>

In [9]:
europeDF.to_sql('countries', dbConnection, if_exists='replace', index = False)
euDF.to_sql('eu', dbConnection, if_exists='replace', index = False)

NameError: name 'dbConnection' is not defined

In [10]:
dbCursor = dbConnection.cursor()

query = """
    SELECT *
    FROM countries
    INNER JOIN eu
    ON countries.Code = eu.Code
"""

result = dbCursor.execute(query).fetchall()
print(result)
print(type(result))

NameError: name 'dbConnection' is not defined

In [ ]:
query = """
    SELECT Code
    FROM countries
"""

result = dbCursor.execute(query).fetchall()
print(result)
print(type(result))

<h3>Exercise 2</h3>
<p>
    Gebruik SQL queries voor het verkrijgen van
<ol>
    <li>het aantal landen dat onderdeel is van de Nordics</li>
    <li>het aantal Europese landen dat lid is van de Europese Unie</li>
    <li>het aantal EU-landen dat deel uitmaakt van de Norics</li>
</ol>
</p>

In [ ]:
# Zoek het aantal landen dat onderdeel is va de Nordics
query = """
    SELECT name
    FROM countries
    WHERE groups = 'nordics'
"""

result = dbCursor.execute(query).fetchall()
print(result)
print(len(result))
print(type(result))


In [ ]:
# Zoek het aantal landen dat onderdeel is van de EU
# De DB countries bevat alle landen van Europa, zonder aan te geven of een land een EU-land is, of niet.
# Door een inner join te maken met de DB  eu, krijg je alleen alle europese landen die ook lid zijn van de eu
# https://www.w3schools.com/SQL/sql_join_inner.asp
# SELECT column_name(s)
# FROM table1
# INNER JOIN table2
# ON table1.column_name = table2.column_name;


query = """
    SELECT countries.name
    FROM countries
    INNER JOIN eu
    ON countries.name = eu.name;
"""
# Typisch: als ik de join maak '0N countries.name = eu.name;' dan krijg ik 26 landen
# maak ik de join 0N countries.Code = eu.Code;' dan krijg ik 27 landen ????
result = dbCursor.execute(query).fetchall()
print(result)
print(len(result))
print(type(result))


In [ ]:
# Zoek het aantal  EU-landen dat deel uitmaakt van de Nordics.
# Oplossing: breidt voorgaande query uit met de beperking dat groups  de waarde 'nordics' heeft

query = """
    SELECT countries.name
    FROM countries
    INNER JOIN eu
    ON countries.name = eu.name
    WHERE groups = 'nordics';

"""

result = dbCursor.execute(query).fetchall()

print(result)
print(len(result))
print(type(result))

In [ ]:
# Close connection to SQLite database
dbConnection.close()